In [1]:
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier as DTC
import os
from sklearn import datasets as ds
from sklearn import model_selection as ms

# This is filtering each file to get pure data , first we remove all the stop words from the file and then filter out the \r the \n and all the stop words from the files to get pure words that work!

In [2]:
stopword={}
s_words=open('stopwords.txt','r')
s_words_data=s_words.read()
for x in s_words_data.split('\n'):
    stopword[x]=1

In [3]:
def createArrayOfStrings(data):
    
    py=data.split(' ')
    arr=[]
    for x in range(len(py)):
        word=''
        word=py[x].replace('\n','')
        word=word.replace('\r','')
        word=word.replace('\t','')
        word=word.replace(',','')
        word=word.replace('(','')
        word=word.replace(').','')
        word=word.replace(')','')
        word=word.replace('.','')
        word=word.replace('"','')    
        word=word.lower()
        if word=='':
            continue
        elif word in stopword.keys():
        
            continue
        else:
            arr.append(word)
    return arr

def filterLabels(labels):
    for x in range(len(labels)):
        labels[x]=labels[x].replace('mini_newsgroups/','')
    return labels

In [4]:
'''path = 'mini_newsgroups/sci.space/'
data=[]
labels=[]
for itemName in os.listdir(path):
    itemName = os.path.join(path, itemName)
    file_readed=open(itemName)
    arr=createArrayOfStrings(file_readed.read())
    data.append(arr)
    labels.append(path)'''

path = 'mini_newsgroups/'
data=[]
labels=[]
for itemName in os.listdir(path):
    itemName = os.path.join(path, itemName)
    print itemName
    for myPath in os.listdir(itemName):
        myPath=os.path.join(itemName,myPath)
        file_readed = open(myPath)
        arr=createArrayOfStrings(file_readed.read())
        data.append(arr)
        labels.append(itemName)
        
     


mini_newsgroups/alt.atheism
mini_newsgroups/comp.graphics
mini_newsgroups/comp.os.ms-windows.misc
mini_newsgroups/comp.sys.ibm.pc.hardware
mini_newsgroups/comp.sys.mac.hardware
mini_newsgroups/comp.windows.x
mini_newsgroups/misc.forsale
mini_newsgroups/rec.autos
mini_newsgroups/rec.motorcycles
mini_newsgroups/rec.sport.baseball
mini_newsgroups/rec.sport.hockey
mini_newsgroups/sci.crypt
mini_newsgroups/sci.electronics
mini_newsgroups/sci.med
mini_newsgroups/sci.space
mini_newsgroups/soc.religion.christian
mini_newsgroups/talk.politics.guns
mini_newsgroups/talk.politics.mideast
mini_newsgroups/talk.politics.misc
mini_newsgroups/talk.religion.misc


In [5]:
# so the data is now a 2d array, and now we are going to create a dictionary for every word in each data
labels=filterLabels(labels)

dictionary={}

for x in data:
    for y in x:
        dictionary[y]=dictionary.get(y,0)+1 #if y not there then place 1 and y in the dictionary as Value and Key resp...

# these are the features for the dataset and we are going to train the GNB on this dataset now!


# remove the elements which have the value 1 in the dictionary
for key in dictionary.keys():
    if dictionary[key]==1:
        del (dictionary[key])

print len(dictionary)

24266


In [6]:
# Create a feature Matrix, where num features are number of words in the dictionary:
feature_length=len(dictionary.keys())
words_dictionary=dictionary.keys()
#creating 2 D matrix now:
X,noUse=np.mgrid[0:len(data), 0:len(dictionary.keys())] #this is a 2d matrix that we will now use
for i in range(len(data)):
    #this is string of one document, filtered of stop words, create a dictionary of words
    d={}
    for word in data[i]:
        d[word]=d.get(word,0)+1
        
    for j in range(X.shape[1]): # in range X-> [ columns ]
        X[i][j]=d.get(words_dictionary[j],0) # say, first word in 'Hello'. Now if 'Hello' exist 2 times in d{}. So this places 2 in X[0][0] 

In [7]:
X_train,X_test,Y_train,Y_test=ms.train_test_split(X,labels,test_size=0.1)

In [49]:
# From Scratch implementation of Naive bayes classifier

def createDictionary(data,labels):
     
    dictionary={}
    for x in data:
        for y in x:
            dictionary[y]=dictionary.get(y,0)+1 #if y not there then place 1 and y in the dictionary as Value and Key resp...

    # these are the features for the dataset and we are going to train the GNB on this dataset now!
    # remove the elements which have the value 1 in the dictionary
    for key in dictionary.keys():
        if dictionary[key]==1:
            del (dictionary[key])
    print 'created dictionary successfully'
    return dictionary

def createX(dictionary,data):
    feature_length=len(dictionary.keys())
    words_dictionary=dictionary.keys()
    #creating 2 D matrix now:
    X,noUse=np.mgrid[0:len(data), 0:len(dictionary.keys())] #this is a 2d matrix that we will now use
    for i in range(len(data)):
        #this is string of one document, filtered of stop words, create a dictionary of words
        d={}
        for word in data[i]:
            d[word]=d.get(word,0)+1

        for j in range(X.shape[1]): # in range X-> [ columns ]
            X[i][j]=d.get(words_dictionary[j],0) # say, first word in 'Hello'. Now if 'Hello' exist 2 times in d{}. So this places 2 in X[0][0] 
    print 'created X successfully'
    return X

#given data, a 2d array of words and the corresponding classes of the data
def fit(data,labels):
    
    dictionary=createDictionary(data,labels)
    labels=filterLabels(labels)         
    # Create a feature Matrix, where num features are number of words in the dictionary:
    X=createX(dictionary,data)
    print 'fit ran perfectly'
    return X,dictionary

def index(word,dictionary):
    keys=dictionary.keys()
    for i in range(len(dictionary.keys())):
        if keys[i]==word:
            return i
    return -1
    
def createX_Counts(X,labels,dictSize):
    X_counts=np.zeros((len(set(labels)),dictSize))
    i=0
    for y in set(labels):
        prob=0
        #-------------------------------------
        arr=np.zeros(len(labels))
        for i_ in range(len(labels)):
            if labels[i_]==y:
                arr[i_]=1
        x_=np.array(arr, dtype=bool)
        # -----------------------------------
        X_small=X[x_]
        for j in range(dictSize):
            X_counts[i][j]=X_small[:,j].sum()
        
        i+=1
    print len(X_counts)
    return X_counts


def predictOne(x,X_counts,labels,dictionary):
    max_prob=0
    class_prob=''
    current_class=0
    
    for i in range(len(set(labels))):
        prob=0
        
        #countAllWordsOfClass=sum(map(sum, X_small))
        print '1'
        for word in (x):
            if index(word,dictionary)!=-1:
                prob+=np.log(X_counts[i,index(word,dictionary)] + 1)#- np.log(countAllWordsOfClass)
        prob=prob+np.log(1.0/20.0) #because prob of each class is 1/20
        if prob>max_prob:
            max_prob=prob
            class_prob=list(set(labels))[i]
    return class_prob

def predict(X_test,X,labels,dictionary):
    ypred=[]
    for x in X_test:
        pred=predictOne(x,X,labels,dictionary)
        print pred
        ypred.append(pred)
    return ypred



In [51]:
#X,dictionary=fit(data,labels)
X_counts= createX_Counts(X,labels,len(dictionary))

print predict(data[1531:1533],X_counts,labels,dictionary)
#run()   


20
1


C:\Users\80LM0141IH\Anaconda3\envs\py27\lib\site-packages\ipykernel_launcher.py:84: RuntimeWarning: divide by zero encountered in log


1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
soc.religion.christian
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
soc.religion.christian
['soc.religion.christian', 'soc.religion.christian']


NameError: name 'ypred' is not defined

In [47]:
a,b=np.mgrid[0:5,0:5]
b.sum()
list(set(labels))[1]


'comp.sys.mac.hardware'